### *Basic information on these data sets can be viewed in 'Data Summary.ipynb'*
#### Features inspired by others' Instacart kernel contributions*

In [1]:
import sys
print (sys.version)

2.7.14 |Anaconda, Inc.| (default, Oct 15 2017, 03:34:40) [MSC v.1500 64 bit (AMD64)]


In [2]:
import os
import numpy as np
import pandas as pd
pd.set_option('precision',4)

### Create User Order Features for Prior Orders

In [6]:
# Combine order and order_products_prior datasets
orders_df = pd.read_csv('input/orders.csv')
order_products_prior_df = pd.read_csv('input/order_products__prior.csv')
prior_order_features_df = orders_df.merge(order_products_prior_df, on='order_id')
prior_order_features_df.sample(10)

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered
8400696,1601916,53269,prior,3,0,12,14.0,22695,8,0
9909156,2113840,62998,prior,13,1,11,21.0,9387,18,1
6877908,1178814,43656,prior,29,2,9,13.0,2295,17,1
12379246,1748031,78321,prior,7,1,9,15.0,3957,1,1
23733109,237476,150746,prior,18,6,19,14.0,22025,11,1
27898939,1577813,177274,prior,14,2,16,15.0,30489,8,1
5981747,763967,38079,prior,3,5,10,30.0,23203,16,0
15039016,1398726,95182,prior,2,5,12,26.0,27966,3,1
18633263,1812658,117925,prior,5,6,14,7.0,30010,12,0
23846265,398222,151419,prior,11,0,20,7.0,42592,8,1


In [10]:
prior_order_features_df['user_orders'] = prior_order_features_df.groupby('user_id')['order_id'].transform('count')
prior_order_features_df['basket_size'] = prior_order_features_df.groupby(['user_id','order_id'])['product_id'].transform('count')
prior_order_features_df['avg_basket_size'] = prior_order_features_df.groupby('user_id')['basket_size'].transform(np.mean)
prior_order_features_df['avg_days_between_orders'] = prior_order_features_df.groupby('user_id')['days_since_prior_order'].transform(np.mean)
prior_order_features_df['avg_add_to_cart_order'] = prior_order_features_df.groupby(['user_id','product_id'])['add_to_cart_order'].transform(np.mean)
prior_order_features_df['user_prod_reorders'] = prior_order_features_df.groupby(['user_id','product_id'])['reordered'].transform(np.sum)
prior_order_features_df['user_prod_reorder_rate'] = ( prior_order_features_df.user_prod_reorders / prior_order_features_df.user_orders )
prior_order_features_df['prod_order_count'] = prior_order_features_df.groupby('product_id')['add_to_cart_order'].transform('count')
prior_order_features_df['prod_reorders'] = prior_order_features_df.groupby('product_id')['reordered'].transform(np.sum)
prior_order_features_df['prod_reorder_rate'] = ( prior_order_features_df.prod_reorders / prior_order_features_df.prod_order_count )

In [19]:
prior_order_features_df.sample(10)

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,user_orders,basket_size,avg_basket_size,avg_days_between_orders,avg_add_to_cart_order,user_prod_reorders,user_prod_reorder_rate,prod_order_count,prod_reorders,prod_reorder_rate
7178992,204026,45558,prior,5,3,13,22.0,10151,7,0,74,21,16.8919,19.7846,7.0000,0,0.0000,4726,2606,0.5514
196417,2899228,1332,prior,2,2,16,22.0,39374,6,0,119,7,12.0924,17.6339,4.6667,2,0.0168,1923,470,0.2444
29128379,1688806,185026,prior,1,0,19,NaN,33198,8,0,21,9,8.8095,30.0000,5.0000,1,0.0476,42934,31500,0.7337
1174718,1449745,7600,prior,16,5,17,7.0,13083,4,1,473,7,9.8161,7.0086,6.8000,4,0.0085,7562,4904,0.6485
15695307,2594789,99302,prior,6,2,8,30.0,25544,1,1,74,9,11.2973,24.6250,1.0000,3,0.0405,6002,2745,0.4573
30649698,3329715,194682,prior,10,4,7,21.0,35939,4,1,128,4,10.6406,15.7845,4.7500,3,0.0234,20262,11462,0.5657
16540817,2376087,104661,prior,2,0,16,26.0,38383,3,0,45,24,18.2444,26.0000,3.0000,0,0.0000,23641,14793,0.6257
732857,3282077,4807,prior,59,3,23,3.0,34358,1,1,629,6,8.3355,3.2051,3.0000,3,0.0048,36617,21510,0.5874
25254564,2133987,160310,prior,54,4,11,6.0,48559,18,1,1207,27,22.7167,6.1002,13.0000,5,0.0041,8910,6448,0.7237
29241848,3382150,185727,prior,34,5,9,6.0,22935,20,1,808,25,18.5891,6.9975,15.5000,1,0.0012,113426,79072,0.6971


In [18]:
prior_order_features_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32434489 entries, 0 to 32434488
Data columns (total 20 columns):
order_id                   int64
user_id                    int64
eval_set                   object
order_number               int64
order_dow                  int64
order_hour_of_day          int64
days_since_prior_order     float64
product_id                 int64
add_to_cart_order          int64
reordered                  int64
user_orders                int64
basket_size                int64
avg_basket_size            float64
avg_days_between_orders    float64
avg_add_to_cart_order      float64
user_prod_reorders         int64
user_prod_reorder_rate     float64
prod_order_count           int64
prod_reorders              int64
prod_reorder_rate          float64
dtypes: float64(6), int64(13), object(1)
memory usage: 6.3+ GB


In [17]:
# check for null or nan values
prior_order_features_df.isnull().sum()

order_id                         0
user_id                          0
eval_set                         0
order_number                     0
order_dow                        0
order_hour_of_day                0
days_since_prior_order     2078068
product_id                       0
add_to_cart_order                0
reordered                        0
user_orders                      0
basket_size                      0
avg_basket_size                  0
avg_days_between_orders          0
avg_add_to_cart_order            0
user_prod_reorders               0
user_prod_reorder_rate           0
prod_order_count                 0
prod_reorders                    0
prod_reorder_rate                0
dtype: int64

In [25]:
# save a sample of 1% of records or approximatly 30K excluding NaN records  
prior_order_features_df = prior_order_features_df.sample(frac=0.001).dropna()
prior_order_features_df.shape

(30369, 21)

In [26]:
# Write to file
prior_order_features_df.to_csv('transformed/prior_order_features.csv')

### Evaluate Feature Importance

In [27]:
# Utilize RFC to evaluate feature predictive importance
from sklearn.ensemble import RandomForestClassifier

y = prior_order_features_df['reordered']
X = prior_order_features_df.drop(['reordered','order_id','user_id','eval_set'],axis=1)

cls = RandomForestClassifier(n_estimators=50)
cls.fit(X, y)

features = X.columns
feature_importances = cls.feature_importances_

features_df = pd.DataFrame({'features': features, 'importance': feature_importances})
features_df.sort_values('importance', inplace=True, ascending=False)

features_df.head()

,features,importance
13,user_prod_reorder_rate,0.3344
12,user_prod_reorders,0.2431
1,order_number,0.0738
16,prod_reorder_rate,0.0391
7,user_orders,0.0345


### We now have or top 5 features of significance using RFC

In [ ]:
# let's try Recursive feature elimination with cross-validation
# http://scikit-learn.org/stable/auto_examples/feature_selection/plot_rfe_with_cross_validation.html

import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import RFECV

y = prior_order_features_df['reordered']
X = prior_order_features_df.drop(['reordered','order_id','user_id','eval_set'],axis=1)

svc = SVC(kernel="linear") # Create the RFE object and compute a cross-validated score.

# The "accuracy" scoring is proportional to the number of correct classifications
rfecv = RFECV(estimator=svc, step=1, cv=StratifiedKFold(2), scoring='accuracy')
rfecv.fit(X, y)

print("Optimal number of features : %d" % rfecv.n_features_)

# Plot number of features VS. cross-validation scores
plt.figure()
plt.xlabel("# Features")
plt.ylabel("Cross Val / Precision Score")
plt.plot(range(1, len(rfecv.grid_scores_) + 1), rfecv.grid_scores_)
plt.show()